In [ ]:
import pandas as pd
df = pd.read_csv("drive/MyDrive/data exercises/train.csv")
df

,id,qid1,qid2,question1,question2,is_duplicate
0,0,1,2,What is the step by step guide to invest in sh...,What is the step by step guide to invest in sh...,0
1,1,3,4,What is the story of Kohinoor (Koh-i-Noor) Dia...,What would happen if the Indian government sto...,0
2,2,5,6,How can I increase the speed of my internet co...,How can Internet speed be increased by hacking...,0
3,3,7,8,Why am I mentally very lonely? How can I solve...,Find the remainder when [math]23^{24}[/math] i...,0
4,4,9,10,"Which one dissolve in water quikly sugar, salt...",Which fish would survive in salt water?,0
...,...,...,...,...,...,...
404285,404285,433578,379845,How many keywords are there in the Racket prog...,How many keywords are there in PERL Programmin...,0
404286,404286,18840,155606,Do you believe there is life after death?,Is it true that there is life after death?,1
404287,404287,537928,537929,What is one coin?,What's this coin?,0
404288,404288,537930,537931,What is the approx annual cost of living while...,I am having little hairfall problem but I want...,0


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# Account for imbalance in data set

from sklearn.model_selection import train_test_split
df_train_0, df_test_0 = train_test_split(df[df['is_duplicate'] == 0], train_size=0.7,random_state=0)
df_train_1, df_test_1 = train_test_split(df[df['is_duplicate'] == 1], train_size=0.7,random_state=0)

df_train = pd.concat([df_train_0, df_train_1]).sample(frac=.5)
df_test = pd.concat([df_test_0, df_test_1]).sample(frac=.5)
y_train = df_train['is_duplicate']
y_test = df_test['is_duplicate']
print(y_test.shape)
print(y_train.shape)

# Make sure no rows got dropped
print(len(df_train)+len(df_test)==len(df))
df_test.head()

(60644,)
(141501,)
False


,id,qid1,qid2,question1,question2,is_duplicate
24031,24031,44955,44956,How do you install fonts in Debian Xfce using ...,Is Freelancer97.com trust worthy?,0
350447,350447,479181,48473,Can you get back deleted comments on Instagram?,Is there a way to get deleted Instagram pictur...,0
76623,76623,130958,130959,Why did Windows defeat all other OSes like Mac...,Why do some programmers prefer Linux OS instea...,0
38865,38865,8908,70523,Are people just being ignorant when they ask h...,When people say the best business are built on...,0
249880,249880,363618,363619,How do I get my passport in only two documents?,How much time can record video continuously wi...,0


# Forest Models

In [ ]:
from nltk.tokenize import word_tokenize
import string
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

In [ ]:
# Forest models
    # Data prep using .itertuples()
# %%time
import nltk
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('omw-1.4')
nltk.download('stopwords')
def process_doc(doc):

    # remove special characters
    doc = ''.join([char for char in str(doc) if not char in string.punctuation]) ###

    # Split text into single words (also gets rid of extra white spaces)
    words = word_tokenize(doc)

    # Convert to lower case
    words = [word.lower() for word in words]

    # Stem
    # porter = PorterStemmer()
    # words = [porter.stem(word) for word in words]
    wnl = WordNetLemmatizer()
    words = [wnl.lemmatize(word) for word in words]

    return words

# feature engineering
def process_pairs(df):
    """
    Perform feature engineering on pairs of questions for forest models.
    """

    length_diff = []
    n_common_words = []
    df2 = pd.DataFrame()
    for index, q1, q2 in df[['question1', 'question2']].itertuples():
        # Preprocess
        tokens1 = process_doc(q1)
        tokens2 = process_doc(q2)
        # print(tokens1)
        # print(tokens2)
        # print()

        # Get number of words for q1 and a2 and the ratio of these values
        df2.loc[index,'Q1 length'] = len(tokens1)
        df2.loc[index,'Q2 length'] = len(tokens2)
        df2.loc[index,'length ratio'] = min([len(tokens1), len(tokens2)]) / max([len(tokens1), len(tokens2)])

        # Number of common words between q1 and q2
        common_words = set(tokens1) & set(tokens2)
        df2.loc[index,'N common words'] = len(common_words)

        # Common words between q1 and q2 as a percentage of longest question in the pair
        df2.loc[index,'common words percentage'] = len(common_words) / max([len(tokens1), len(tokens2)])
        try:
            df2.loc[index,'same last word'] = int(tokens1[-1] == tokens2[-1])
        except:
            df2.loc[index,'same last word'] = 0

        # Same frequency of the word 'not'
        df2.loc[index,'not_count1'] = sum(word=='not' for word in tokens1)
    
        # # Remove stop words aside from the 5 why's + how
        stop_words = set(stopwords.words('english')) - set(['who', 'what', 'where', 'when', 'how', 'why'])
        words1 = [w for w in tokens1 if not w in stop_words]
        words2 = [w for w in tokens2 if not w in stop_words]

        # Number of common words between q1 and q2 with stop words removed
        common_words_nonstop = set(words1) & set(words2)
        df2.loc[index,'N common non-stop words'] = len(common_words_nonstop)
        df2.loc[index,'common non-stop words percentage'] = len(common_words_nonstop) / max(
            [len(words1), len(words2)])
        
        # print(words2,'\n')
        
    # length difference
    df2['Q1 Q1 length difference'] = abs(df2['Q1 length'] - df2['Q2 length'])

    return df2
        

X_train = process_pairs(df_train)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


KeyboardInterrupt: ignored

In [ ]:
print(type(X_train))
import pickle
with open('drive/MyDrive/data exercises/X_train.sav', 'wb') as fh:
   pickle.dump(X_train, fh)

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.base import BaseEstimator
class ClfSwitcher(BaseEstimator):
# Create a class named ClfSwitcher which inherits the base class called BaseEstimator from sklearn.
    def __init__(self, estimator = LogisticRegression()):
            self.estimator = estimator # receives an estimator (model) as an input
            
    def fit(self, X, y=None, **kwargs):
            self.estimator.fit(X, y)
            return self
            
    def predict(self, X, y=None):
            return self.estimator.predict(X)
            
    def predict_proba(self, X):
            return self.estimator.predict_proba(X)
            
    def score(self, X, y):
            return self.estimator.score(X, y)


from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import FunctionTransformer
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
import xgboost as xgb

forest_pipe = Pipeline([
    # ('process_pairs', FunctionTransformer(process_pairs)),
    ('model', ClfSwitcher())
])
# forest_pipe.fit_transform(df.head()) # This line works before the modeling step
grid_params = [
    {
        'model__estimator': [LogisticRegression(random_state=0)],
        'model__estimator__class_weight': [None, 'balanced']
        },
    {
        'model__estimator': [SVC(random_state=0)],
        'model__estimator__class_weight': [None, 'balanced']
        },
    {
        'model__estimator': [xgb.XGBClassifier(random_state=0)],
        'model__estimator__n_estimators': [100, 150, 200]
        },
    {
        'model__estimator': [RandomForestClassifier(random_state=0)],
        'model__estimator__n_estimators': [100, 150, 200]
        },
    ]
gs = GridSearchCV(forest_pipe, grid_params, scoring='accuracy')
y_pred_train = gs.fit(X_train, y_train)
# print(gs.best_estimator_)
print(gs.best_params_)

KeyboardInterrupt: ignored

In [ ]:
from sklearn.metrics import ConfusionMatrixDisplay, RocCurveDisplay, accuracy_score, f1_score, precision_score, recall_score, roc_auc_score

def evaluate_classifier(classifier, X_test, y_test, X_train, y_train, pos_label=1,model_name='classifier', confusion_matrix=True):
    """
    Print evaluation metrics of the classifier. Classifier should be fit prior to calling this function.
        * recall
        * precision
        * F1
        * AUC score (only if class labels are integers)
    Plot:
        * confusion matrix
        * ROC (only if class labels are integers)
    Parameters:
    - pos_label (str or int): Class label for positive class. Default is 1.
    - model_name (string, optional): Name of model printing purposes.
    - confusion_matrix (bool): If True (default), print the confusion matrix. 

    Returns 2 dictionaries:
    - evaluation metrics for test data
    - evaluation metrics train data set
    """
    
    best_model = classifier

    y_pred = best_model.predict(X_test)

    y_pred_train = best_model.predict(X_train)

    # Metrics for test data
    metrics = dict()
    if len(set(y_test)) == 2:
        recall = recall_score(y_test, y_pred, pos_label=pos_label)
        precision = precision_score(y_test, y_pred, pos_label=pos_label)
        f1score = f1_score(y_test, y_pred, pos_label=pos_label)
        metrics['recall'] = recall
        metrics['precision'] = precision
        metrics['f1'] = f1score

    accuracy = accuracy_score(y_test, y_pred)
    metrics['accuracy'] = accuracy

    # Metrics for training data
    metrics_train = dict()
    if len(set(y_test)) == 2:
        recall_train = recall_score(y_train, y_pred_train, pos_label=pos_label)
        precision_train = precision_score(y_train, y_pred_train, pos_label=pos_label)
        f1score_train = f1_score(y_train, y_pred_train, pos_label=pos_label)
        metrics_train['recall'] = recall_train
        metrics_train['precision'] = precision_train
        metrics_train['f1'] = f1score_train
    accuracy_train = accuracy_score(y_train, y_pred_train)
    metrics_train['accuracy'] = accuracy_train

    print(f'\n{model_name} evaluation metrics: \n\tTest data\tTraining data\t\tDifference')
    print(f'Accuracy: \t{100*accuracy:.2f}%\t\t{100*accuracy_train:.2f}%\t\t{100*(accuracy-accuracy_train):.2f}%')
    if len(set(y_test)) == 2:
        print(f'Recall: \t{100*recall:.2f}%\t\t{100*recall_train:.2f}%\t\t{100*(recall-recall_train):.2f}%')
        print(f'Precision: \t{100*precision:.2f}%\t\t{100*precision_train:.2f}%\t\t{100*(precision-precision_train):.2f}%')
        print(f'F1: \t\t{100*f1score:.2f}%\t\t{100*f1score_train:.2f}%\t\t{100*(f1score-f1score_train):.2f}%')
        if type(y_test) == 'int':
            auc = roc_auc_score(y_test, y_pred)
            auc_train = roc_auc_score(y_train, y_pred_train)
            print(f'AUC: \t\t{100*auc:.2f}%\t\t{100*auc_train:.2f}%\t\t{100*(auc-auc_train):.2f}%')
            RocCurveDisplay.from_estimator(best_model, X_train, y_train)
            metrics['auc'] = auc
    if confusion_matrix==True:
        ConfusionMatrixDisplay.from_estimator(best_model, X_train, y_train)
    return metrics, metrics_train

In [ ]:
metrics, metrics_train = evaluate_classifier(gs.best_estimator_, df_test, y_test, df_train, y_train, pos_label=1,model_name='RandomForestClassifier')

TypeError: ignored